In [1]:
import torch
import torchvision as tv

In [19]:
samples0, samples1 = 60000, 10000
# features = 784
classes = 10

source0 = tv.datasets.MNIST("../../MNIST", train = True, download = False)
source1 = tv.datasets.MNIST("../../MNIST", train = False, download = False)
DATA0 = source0.data.unsqueeze(1).float().cuda()
DATA1 = source1.data.unsqueeze(1).float().cuda()
TARGET0 = source0.targets.cuda()
TARGET1 = source1.targets.cuda()

BIAS1 = torch.zeros(8, 1, 1, requires_grad=True, device="cuda")
FILTER1 = torch.zeros(8, 1, 5, 5, requires_grad = True, device = "cuda")
BIAS = torch.zeros(1, classes, requires_grad = True, device = "cuda")
WEIGHT = torch.zeros(8*24*24, classes, requires_grad = True, device = "cuda")
torch.nn.init.xavier_normal_(FILTER1)
variables = [BIAS, WEIGHT, FILTER1, BIAS1]

In [17]:
def model(SIGNAL):
    SIGNAL = BIAS1 + torch.nn.functional.conv2d(SIGNAL, FILTER1)
    SIGNAL = torch.nn.functional.relu(SIGNAL)
    SIGNAL = SIGNAL.flatten(1)
    
    SIGNAL = BIAS + SIGNAL @ WEIGHT
    return SIGNAL

In [18]:
batch = 1000
optimizer = torch.optim.Adam(variables, lr = 0.0001)
for epoch in range(10):
    LOSS0 = torch.zeros((), device = "cuda")
    ACCURACY0 = torch.zeros((), device = "cuda")
    count0 = 0
    for index in range(0, samples0, batch):
        optimizer.zero_grad()
        DATA = DATA0[index : index + batch]
        TARGET = TARGET0[index : index + batch]
        count = TARGET.size(0)
        ACTIVATION = model(DATA)
        LOSS = torch.nn.functional.cross_entropy(ACTIVATION, TARGET)
        LOSS0 += LOSS * count
        VALUE = torch.argmax(ACTIVATION, 1)
        ACCURACY0 += torch.sum(VALUE == TARGET)
        count0 += count
        LOSS.backward()
        optimizer.step()
    LOSS0 /= count0
    ACCURACY0 /= count0
    with torch.no_grad():
        LOSS1 = torch.zeros((), device = "cuda")
        ACCURACY1 = torch.zeros((), device = "cuda")
        count1 = 0
        for index in range(0, samples1, batch):
            DATA = DATA1[index : index + batch]
            TARGET = TARGET1[index : index + batch]
            ACTIVATION = model(DATA)
            LOSS1 += torch.nn.functional.cross_entropy(ACTIVATION, TARGET, reduction = "sum")
            VALUE = torch.argmax(ACTIVATION, 1)
            ACCURACY1 += torch.sum(VALUE == TARGET)
            count1 += TARGET.size(0)
        LOSS1 /= count1
        ACCURACY1 /= count1
    print("%5d %12.3f %4.3f %12.3f %4.3f" % \
          (epoch, LOSS0, ACCURACY0, LOSS1, ACCURACY1), flush = True)

#accuracy: train: 931 test: 920

    0        0.383 0.885        0.188 0.948
    1        0.157 0.956        0.135 0.963
    2        0.108 0.970        0.107 0.970
    3        0.082 0.977        0.094 0.972
    4        0.065 0.982        0.089 0.973
    5        0.053 0.986        0.083 0.974
    6        0.045 0.988        0.079 0.977
    7        0.039 0.990        0.078 0.977
    8        0.034 0.992        0.077 0.978
    9        0.029 0.993        0.073 0.978
